In [1]:
import requests
import re
import time
import csv

In [6]:
# 实验，API，不可用，等待更新
! curl -G 'https://api.bilibili.com/x/v2/reply/main' \
--data-urlencode 'type=1' \
--data-urlencode 'oid=2' \
--data-urlencode 'mode=3' \
--data-urlencode 'next=0' \
--data-urlencode 'ps=5' \
-b 'SESSDATA=fbda0bdc%2C1636301184%2C30180681'

{"code":-352,"message":"-352","ttl":1}


In [2]:
#时间戳转换成日期
def get_time(ctime):
    timeArray = time.localtime(ctime)
    otherStyleTime = time.strftime("%Y.%m.%d", timeArray)
    return str(otherStyleTime)

#获取aid
def get_aid(bvid):
    video_url = 'https://api.bilibili.com/x/web-interface/view?bvid=' + bvid #最新的获取aid方式
    page = requests.get(video_url, headers=header).text
    aid = re.search(r'"aid":[0-9]+', page).group()[6:]
    return aid

如何更改评论的API，以及获取更多的评论消息：  
参考API网址：[GITHUB: B站API大全](https://github.com/SocialSisterYi/bilibili-API-collect/blob/master/docs/comment/list.md)

In [32]:
#边爬取评论边保存文件
def online_save(Bvid, max_pages:int = 999, fpath:str = '/home/'):
    '''
    这里必要的选项为：BVID —— 提供BVID的号码；
    可选项:
      max_pages: 最大获取的页数，每一页都是10个主评论
      fpath: 文件储存地址，默认为linux中的/home文件夹
    '''
    #消息头信息
    header={
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36',
    }

    #评论API
    api_url = 'https://api.bilibili.com/x/v2/reply'

    #主params
    params = {
      'type': '1', #评论区类型代码，来源：https://github.com/SocialSisterYi/bilibili-API-collect/blob/master/docs/comment/readme.md#%E8%AF%84%E8%AE%BA%E5%8C%BA%E7%B1%BB%E5%9E%8B%E4%BB%A3%E7%A0%81
      'oid': '', #目标评论区 id, 根据aid过来的
      'sort': '1', # 排序方式（非必要，默认为0）  0：按时间 1：按点赞数 2：按回复数
      'nohot': '1', #是否不显示热评（非必要，默认为0）1：不显示  0：显示
      'ps': '10', #每页项数（非必要，默认为20  定义域：1-20）
      'pn': '1' #页码（非必要，默认为1）
    }

    # 更新aid
    params['oid'] = get_aid(Bvid)

    # 获取HTML，换成json
    html = requests.get(url = api_url, headers=header, params=params, cookies=cookies)
    if html.status_code != 200: return 'BVID或者相关信息错误，无法打开HTML界面'
    data = html.json()
    try:
        r_total_count = int(data['data']['page']['acount']) #评论总数
    except:
        return f"无法读取格式，返回值为{data}"


    fname = Bvid + '_评论.csv'

    page = 1
    pointer_count = 0 #页数指示器
    print(f'总评论数{r_total_count}，总共需要爬取{r_total_count//10 + 1}页，开始创建文件')
    with open(fpath+fname, 'w', newline='', encoding='utf_8_sig') as f:
        csv_writer=csv.writer(f)
        csv_writer.writerow(["时间", "点赞", "评论ID","评论"])
        while pointer_count < (r_total_count//10 + 1) and pointer_count < max_pages:
          for i in data['data']['replies']: #每一个大的replies中
            try:
              message=i['content']['message'] #其中的content为回复内容，message为信息
              message = re.sub('\s+', '', message) #去掉空格？
              rpid = i['rpid']
              ctime=get_time(i['ctime']) #时间（包括转换）
              like=i['like'] #点赞数
              csv_writer.writerow([ctime,str(like),rpid,message])
            except Exception as e:
              f.close()
              raise e #任何的错误将会被返回
          print('总评论数：{}，当前评论页数:{},当前Page已爬取完毕。'.format(r_total_count, pointer_count))

          print('开始暂停5s，下一页准备')
          time.sleep(5)

          #开始进行api更新
          params['pn'] = str(int(params['pn']) + 1)
          html = requests.get(url = api_url, headers=header, params=params, cookies=cookies)
          if html.status_code != 200:
            f.close()
            return 'BVID或者相关信息错误，无法打开HTML界面'
          data = html.json()

          pointer_count = pointer_count + 1
        f.close()

In [30]:
if __name__=='__main__':
    Bvid=input('输入视频Bvid:')
    online_save(Bvid, max_pages = 10)
    print('完成！')

输入视频Bvid:BV1L5411J72C
总评论数1836，总共需要爬取184页，开始创建文件
10
总评论数：1836，当前评论页数:0,当前Page已爬取完毕。
开始暂停5s，下一页准备
总评论数：1836，当前评论页数:1,当前Page已爬取完毕。
开始暂停5s，下一页准备
总评论数：1836，当前评论页数:2,当前Page已爬取完毕。
开始暂停5s，下一页准备
总评论数：1836，当前评论页数:3,当前Page已爬取完毕。
开始暂停5s，下一页准备
总评论数：1836，当前评论页数:4,当前Page已爬取完毕。
开始暂停5s，下一页准备
总评论数：1836，当前评论页数:5,当前Page已爬取完毕。
开始暂停5s，下一页准备
总评论数：1836，当前评论页数:6,当前Page已爬取完毕。
开始暂停5s，下一页准备
总评论数：1836，当前评论页数:7,当前Page已爬取完毕。
开始暂停5s，下一页准备
总评论数：1836，当前评论页数:8,当前Page已爬取完毕。
开始暂停5s，下一页准备
总评论数：1836，当前评论页数:9,当前Page已爬取完毕。
开始暂停5s，下一页准备
完成！


In [ ]:
video_url = 'BV1L5411J72C'
page = requests.get(video_url, headers=header).text
aid = re.search(r'"aid":[0-9]+', page).group()[6:]
print(aid)

458865803
